In [1]:
from collections import OrderedDict

In [2]:
# IRL, read from extrernal metadata / "single config"
field_widths = OrderedDict()
field_widths['year']    = 4
field_widths['month']   = 2
field_widths['day']     = 2

In [3]:
import petl as etl

In [4]:
date_parts_table = [['year', 'month', 'day'],
    ['18',   ' 5', ' 2'],
    ['2018', ' 5', ' 2'],
    ['2018', '05', '02']]

In [5]:
etl.wrap(date_parts_table)

year,month,day
18,5,2
2018,5,2
2018,05,02


In [6]:
# `width` could be more than an int, like an object specifying attributes, e.g. left/right alignment
def make_fixed_width_template(**widths):
    template = ''
    for field_name, width in widths.items():
        template += '{' + f'{field_name}:>{width}' + '}'
    template += '\n'
    return template 

In [7]:
fixed_width_template = make_fixed_width_template(**field_widths)

In [8]:
fixed_width_template

'{year:>4}{month:>2}{day:>2}\n'

In [9]:
from pathlib import Path

In [10]:
date_parts_file = Path('date_parts_table.txt')

In [11]:
etl.totext(date_parts_table, date_parts_file , template=fixed_width_template)

In [12]:
print(date_parts_file.read_text())

  18 5 2
2018 5 2
20180502



In [13]:
def fixed_width_field(start, end):
    return lambda r: r.lines[start:end]

In [14]:
def make_fixed_width_fields(**widths):
    fields = OrderedDict()
    start, end = 0, 0
    for field_name, width in widths.items():
        end += width
        fields[field_name] = fixed_width_field(start, end)
        start = end
    return fields

In [15]:
fixed_width_fields = make_fixed_width_fields(**field_widths)

In [16]:
lines_table = etl.fromtext(date_parts_file, strip='\n')

In [17]:
result_table = etl.fieldmap(lines_table, fixed_width_fields)

In [18]:
result_table

year,month,day
18,5,2
2018,5,2
2018,05,02
